In [11]:
from anthropic import Anthropic


client = Anthropic(api_key="")



In [2]:
tool_definition = [
    {
        "name": "create_ec2_instance",
        "description": "Creates an EC2 instance with the specified parameter",
        "input_schema": {
            "type": "object",
            "properties": {
                "instance_type": {
                    "type": "string",
                    "description": "The instance type, e.g., t2.micro, m5.large, etc."
                },
                "ami_id": {
                    "type": "string",
                    "description": "The AMI ID to use for the instance"
                },
                "min_count": {
                    "type": "integer",
                    "description": "The minimum number of instances to launch"
                },
                "max_count": {
                    "type": "integer",
                    "description": "The maximum number of instances to launch"
                },
            },
            "required": ["instance_type", "ami_id", "min_count", "max_count"]
        }
    }
]

def get_claude_response(prompt):
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=1000,
        temperature=0,
        system="You are an AI assistant that helps to create EC2 instances. Extract the necessary parameters from the user's input and use the create_ec2_instance tool to create the instance.",
        messages=[
            {"role": "user", "content": prompt}
        ],
        tools=tool_definition
    )
    return response






In [1]:
import boto3

def create_ec2_instance(instance_type, ami_id, min_count, max_count):
    session = boto3.Session(
        aws_access_key_id='',
        aws_secret_access_key='',
        region_name='us-east-1'
    )

    # Get the EC2 resource
    ec2 = session.resource('ec2')
    instance = ec2.create_instances(
        ImageId=ami_id,  # Replace with your desired AMI ID
        MinCount=1,
        MaxCount=1,
        InstanceType=instance_type,  # Replace with your desired instance type
      
    )
    print("created instance")
    




In [5]:
prompt = "Create an EC2 instance with the instance type t2.micro and the AMI ID ami-0427090fd1714168b. Launch a minimum of 1 instance and a maximum of 1 instance."
response = get_claude_response(prompt)
if response.stop_reason == 'tool_use':
        print("calling tool")
        print(response.content[0].input)
        create_ec2_instance(response.content[0].input['instance_type'], response.content[0].input['ami_id'], response.content[0].input['min_count'], response.content[0].input['max_count'])
        
        

calling tool
{'instance_type': 't2.micro', 'ami_id': 'ami-0427090fd1714168b', 'min_count': 1, 'max_count': 1}
created instance


In [7]:
response.content

[ToolUseBlock(id='toolu_01Ja5FMno4gKfne7SdSJkcnz', input={'instance_type': 't2.micro', 'ami_id': 'ami-0427090fd1714168b', 'min_count': 1, 'max_count': 1}, name='create_ec2_instance', type='tool_use')]

In [9]:
final_response = client.messages.create(
                model="claude-3-haiku-20240307",
                max_tokens=1000,
                temperature=0,
                system="You are an AI assistant that helps create EC2 instances. Summarize the result of the EC2 instance creation.",
                messages=[
                    {"role": "user", "content": prompt},
                    {"role": "assistant", "content": "d"},
                    {"role": "user", "content":  "s"}
                ]
            )

In [10]:
final_response

Message(id='msg_014yHyr5HiD4qgdqVm4nRAoH', content=[TextBlock(text='Here is a summary of the EC2 instance creation:\n\n- Instance Type: t2.micro\n- AMI ID: ami-0427090fd1714168b\n- Minimum Instances: 1\n- Maximum Instances: 1\n\nThe EC2 instance was successfully created with the specified configuration. The new instance is now running and ready for use.', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=88, output_tokens=82))